In [ ]:
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm


### Here I create my Lidar model.

-   To do this I use Pytorch's modules like: nn.Module(), nn.Sequential(), nn.Linear(), etc.
-   To start, I create the class LidarBrain with nn.Module between the ().
-   Then I define my model, I decided to use nn.Linear() since this is a regression problem.
-   I use nn.Sequential() to define the amount of layers and so that I'm able to change the activation function per layer.
-   After some experimentation I've found that using the activation function nn.ReLU() worked best. 

In [ ]:
class LidarBrain(nn.Module):
    
    def __init__(self, inputs_size, hidden1_size, hidden2_size, hidden3_size, hidden4_size, hidden5_size, outputs_size):
        super().__init__()

        self.lb_model = nn.Sequential(
            nn.Linear(inputs_size, hidden1_size),
            nn.ReLU(),
            nn.Linear(hidden1_size, hidden2_size),
            nn.ReLU(),
            nn.Linear(hidden2_size, hidden3_size),
            nn.ReLU(),
            nn.Linear(hidden3_size, hidden4_size),
            nn.ReLU(),
            nn.Linear(hidden4_size, hidden5_size),
            nn.ReLU(),
            nn.Linear(hidden5_size, outputs_size)
        )

    def forward(self, x):

        return self.lb_model(x)

### Load in the data

-   I load in the data and assign X (scanner data) and Y (steering angle). I decided to leave out the target velocity as that would complicate things.
-   I also convert the data to a torch.Tensor(), this worked best after first converting it to a np.array.

In [ ]:
load_path = "../data/data_3rounds"

data_file = pd.read_table(load_path, sep = ",", index_col = 0)


# Create X data and Y data by converting them to torch.tensor
X = torch.Tensor(np.array(data_file.iloc[:, :-2], copy=False))

# We decided to leave out the velocity for now.
Y = torch.Tensor(np.array(data_file.iloc[:, 16:17], copy=False))

print(X)
print(Y[0].type())
print(data_file.head())

### Time to train the model!

-   During this project I've experimented with a lot of different settings (more about this after saving the model).
-   I start by defining the epochs and learning rate

In [ ]:
# Define epochs and learning rate.
epochs = 200
learningRate = 0.000095

# Define model and amount of neurons per layer.
lb = LidarBrain(len(X[0]), 128, 128, 128, 128, 128, len(Y[0]))
print(lb)

# Create empty list for losses
losses = []

# Take binary cross entropy as loss function (one output interpreted as binary)
lossFunction = nn.MSELoss()

# Use stochastic gradient descent as optimizer, use weights and biases of model
gradientDescent = torch.optim.SGD(lb.parameters(), lr=learningRate)

# Add a loader for faster calculation and being able to use the shuffle=True function (which makes it less likely the model will just remember the track in order).
loader = DataLoader(list(zip(X, Y)), shuffle=True, batch_size=10)

# Train. I'm making use of the tqdm() library to keep track while it's buisy
for i in tqdm(range(epochs)):

    # Create empty losses_epoch list so I can .append() per epoch.
    losses_epoch = []

    for x, y in loader:
    
        # Reset the gradient delta's
        gradientDescent.zero_grad()

        # Forward step
        yhat = lb(x)

        # Compute loss
        loss = lossFunction(yhat, y)
        
        # Keep track of loss
        losses_epoch.append(loss.item())

        # Apply gradient descent (via backpropagation)
        loss.backward()

        # Use w += -step * dw * learnRate
        gradientDescent.step()

    # Append losses per epoch
    losses.append(sum(losses_epoch)/len(losses_epoch))

print(losses)

# Plot the losses (cost) vs the amount of epochs
fig, ax = plt.subplots()
ax.set(xlabel='Epoch', ylabel='Cost', title="Training Cost")

plt.plot([x for x in range(len(losses))], losses, 'red')
plt.show()

### save the model.

-   Use pickle to save the model.
-   To experiment and try different epochs/learningrate/hidden-layers etc. I just returned to this notebook every time and saved it again to overwrite the model before or to a new name if I wanted to keep the old model.

In [ ]:
save_path = "../pickle/LidarBrain.pkl"

pickle.dump(lb.lb_model, open(save_path, 'wb'))

## After trying lots and lots of different things my conclusion is that Lidar is to unstable on THIS track to function in a safe way.

Things I've tried:
- Different Epochs
- Different learning rates
- Different data sets
- Different batch sizes
- More/less hiddenlayers
- More/less neurons
- With/without activation function.
- Different max range on the scanner
- Change the data by applying the IQR method


### Now with Sonar

-   Since I did lidar first, all I had to do was do the exact same thing with sonar. As I set the input in my model as len(X) all I had to do was copy and paste the lidar version and change a few things so that it is it's own thing.

In [ ]:
class SonarBrain(nn.Module):
    
    def __init__(self, inputs_size, hidden1_size, hidden2_size, outputs_size):
        super().__init__()

        self.sb_model = nn.Sequential(
            nn.Linear(inputs_size, hidden1_size),
            nn.ReLU(),
            nn.Linear(hidden1_size, hidden2_size),
            nn.ReLU(),
            nn.Linear(hidden2_size, outputs_size)
        )

    def forward(self, x):

        return self.sb_model(x)

In [ ]:
load_path = "../data/sonar_data_3r"

data_file_2 = pd.read_table(load_path, sep = ",", index_col = 0)


# Create X data and Y data by converting them to torch.tensor
X_2 = torch.Tensor(np.array(data_file_2.iloc[:, :-2], copy=False))

# We decided to leave out the velocity for now.
Y_2 = torch.Tensor(np.array(data_file_2.iloc[:, 3:4], copy=False))

print(X_2)
print(Y_2)
print(data_file_2.head())

In [ ]:
epochs_2 = 100

learningRate_2 = 0.00001

#----------------------------------------

sb = SonarBrain(len(X_2[0]), 64, 64, len(Y_2[0]))

print(sb)

#----------------------------------------

losses = []

# Take binary cross entropy as loss function (one output interpreted as binary)
lossFunction_2 = nn.MSELoss()

# Use stochastic gradient descent as optimizer, use weights and biases of model
gradientDescent_2 = torch.optim.SGD(sb.parameters(), lr=learningRate_2)

#----------------------------------------

for j in tqdm(range(epochs_2)):

    losses_epoch = []
    for x, y in zip(X_2, Y_2):
    
        # Reset the gradient delta's (dw, db)
        gradientDescent_2.zero_grad()

        # Forward step
        yhat = sb(x)

        # Compute loss
        loss = lossFunction_2(yhat, y)
        
        # Keep track of loss
        losses_epoch.append(loss.item())

        # Apply gradient descent (via backpropagation)
        loss.backward()

        # Use w += -step * dw * learnRate
        gradientDescent_2.step()

    losses.append(sum(losses_epoch)/len(losses_epoch))

print(losses)

#----------------------------------------

fig, ax = plt.subplots()
ax.set(xlabel='Epoch', ylabel='Cost', title="Training Cost")

plt.plot([x for x in range(len(losses))], losses, 'red')
plt.show()

#----------------------------------------

In [ ]:
save_path = "../pickle/SonarBrain.pkl"

pickle.dump(sb.sb_model, open(save_path, 'wb'))

# Conclusion

-   Experimenting with both Lidar and Sonar i've found that Lidar is very unstable compared to Sonar, my sonar model worked almost perfectly after 2 different training sessions while I've worked for almost a whole week to try and make the Lidar version more stable.

-   After testing the hardcoded Lidar version some more I found out that after a couple of rounds it also straightup runs off the track. This means I've been training my Lidar model with unstable data to begin with and it won't ever be stable using this type of machinelearning.